In [ ]:
# Install the pytorch and ultralytics 
%pip install -U ultralytics
%pip install torch torchvision torchaudio
%pip install pandas

In [5]:
from ultralytics import YOLO
import ultralytics
import pandas as pd
from distutils.dir_util import copy_tree
import os
# Check  GPU  and pytorch works
ultralytics.checks()


Ultralytics 8.3.1 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
Setup complete ✅ (16 CPUs, 31.1 GB RAM, 500.3/915.3 GB disk)


In [6]:
model = YOLO("./T22without_pipeline.pt")



In [7]:
from tkinter.filedialog import askdirectory

#Use GUI Select Folder
# Put the images you want to predict in the folder
path = askdirectory(title='Select Folder')
print(path.split('/')[-1])


result = model.predict(path,save = True, save_txt = True ,save_conf = True, show_conf = False )




#model.predict('')
# Example usage
#path_to_search = "runs/obb/"
#newest_folder = get_newest_folder(path_to_search)

#labels = [os.path.join(path_to_search+newest_folder+'/labels', x) for x in os.listdir(path_to_search+newest_folder+'/labels') if x[-3:] == "txt"]
#labels.sort()



images



/home/ray/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


image 1/1 /home/ray/BerlinHUYoloStomata/images/T16L600_W1_GC1_R2_P74_g55_3.jpg: 768x1024 126.6ms
Speed: 6.8ms preprocess, 126.6ms inference, 203.3ms postprocess per image at shape (1, 3, 768, 1024)
Results saved to /home/ray/BerlinHUYoloStomata/runs/obb/predict
1 label saved to /home/ray/BerlinHUYoloStomata/runs/obb/predict/labels



# Converting YOLOv8 OBB Format `.txt` Files to `.csv`

This guide explains how to convert YOLOv8 Oriented Bounding Box (OBB) format `.txt` files into a `.csv` file format for easier analysis or processing.

## YOLOv8 OBB `.txt` File Format

In the YOLOv8 OBB `.txt` format, each line in the file corresponds to a detected object in an image, with the following structure:

Where:
- `class_id`: The class label of the detected object.
- `x1, y1`, `x2, y2`, `x3, y3`, `x4, y4`: The coordinates of the four corners of the oriented bounding box.
- `confidence`: The confidence score of the detection.


Convert to
- `center x` , `center y` : The center of the rotated bounding box
- `w` , `h` : the width and height of rotated bounding box  
- `angle` : the angle of rotate

The final output file is Result.csv

In [8]:

import os
import pandas as pd
import math
import numpy as np

# Function to convert the coordinate to cx ,cy , h , w
def get_angle(x1,y1,x2,y2,x3,y3,x4,y4):
    if x2-x1 != 0 and x4-x3 != 0:
        theta = (math.atan((y2-y1)/(x2-x1)) + math.atan((y4-y3)/(x4-x3)))/2
    else:
        theta = np.pi/2

    if theta >= 0:
        angle = theta
    else:
        angle = theta + np.pi
    return angle,theta


def get_w_and_h(x1,y1,x2,y2,x3,y3,x4,y4):
    w = math.sqrt((x2-x1)**2+(y2-y1)**2)
    h = math.sqrt((x1-x3)**2+(y1-y3)**2)
    return w,h

def get_x_and_y(x1,y1,x2,y2,x3,y3,x4,y4,theta,w,h):
    cx = (x2+x3)/2
    cy = (y2+y3)/2
    x = cx - w/2
    y = cy - h/2
    return x,y,cx,cy


labels = [os.path.join('./runs/obb/predict/labels', x) for x in os.listdir('./runs/obb/predict/labels') if x[-3:] == "txt"]
#labelsname = [f for f in os.listdir('./runs/obb/predict/labels')]

labels.sort()
#labelsname.sort()

# Convert and save the annotations

class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                           "blurry.complete": 2,
                           "blurry.incomplete" :3 ,
                           "hair":4
                           } # human hair

id_to_class_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))


df = pd.DataFrame(columns=[ 'class' ,'boundingbox_x' , 'boundingbox_y','boundingbox_width' ,'boundingbox_height' , 'angle' , 'confidence', 'File Name' ])

# Read txt label file
for name in labels:
    fr_line = open(name)
    for bbox_line in fr_line.readlines():
            info = bbox_line.strip().split(' ')
            cls= int(info[0])
            x1,y1,x2,y2,x3,y3,x4,y4 = float(info[1]) , float(info[2]) , float(info[3]) , float(info[4]) , float(info[5]) , float(info[6]) , float(info[7]) , float(info[8])
            conf = float(info[9])
            file_name = name.split('/')[-1]
            angle,theta = get_angle(x1,y1,x2,y2,x3,y3,x4,y4)
            w,h = get_w_and_h(x1,y1,x2,y2,x3,y3,x4,y4)
            x,y,cx,cy = get_x_and_y(x1,y1,x2,y2,x3,y3,x4,y4,theta,w,h)
            new_row = {'class': id_to_class_name_mapping[cls] , 'boundingbox_x' : cx, 'boundingbox_y' : cy , 'boundingbox_width' : w, 'boundingbox_height' : h, 'angle': angle, 'confidence' : conf , 'File Name' : file_name }
            df.loc[len(df)] = new_row






# Save the Dataframe to csv file
#save_name = './Results.csv'
df.to_csv( './result/Ntu/'+path.split('/')[-1]+'.csv', encoding = 'utf-8' ,index = False)